In [5]:
from datasets import load_dataset

dataset = load_dataset("Whispering-GPT/lex-fridman-podcast")

/Users/pattssun/Library/CloudStorage/GoogleDrive-patricksun8530@gmail.com/My Drive/LAB/podcast-chatbot/podcast-chatbot/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
import pandas as pd
import numpy as np
import openai
import faiss
import os

In [7]:
from openai import OpenAI

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)

In [8]:
# Convert the "train" split to a pandas DataFrame
df = pd.DataFrame(dataset["train"])

print(df.head())


            id      channel                channel_id  \
0  TRdL6ZzWBS0  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   
1  TPXTmVdlyoc  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   
2  -t1_ffaFXao  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   
3  BCdV6BMMpOo  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   
4  Kedt2or9xlo  Lex Fridman  UCSHZKyawb77ixDdsGog4iWA   

                                               title              categories  \
0  Jed Buchwald: Isaac Newton and the Philosophy ...  [Science & Technology]   
1  Sergey Nazarov: Chainlink, Smart Contracts, an...  [Science & Technology]   
2  Stephen Wolfram: Fundamental Theory of Physics...  [Science & Technology]   
3  Philip Goff: Consciousness, Panpsychism, and t...  [Science & Technology]   
4  Oriol Vinyals: DeepMind AlphaStar, StarCraft, ...  [Science & Technology]   

                                                tags  \
0  [agi, ai, ai podcast, artificial intelligence,...   
1  [agi, ai, ai podcast, artificial intelligence,...   
2  [step

In [11]:
def get_embedding(text):
    response = openai.embeddings.create(input=[text], model="text-embedding-ada-002")
    return np.array(response['data'][0]['embedding'])

In [12]:
# Get embedding for text of the first row
embedding_first_row = get_embedding(df.iloc[0]['text'])

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 18397 tokens (18397 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}